# Description

Understand rolling window functions

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html

# Imports

In [12]:
%load_ext blackcellmagic

In [1]:
import pandas as pd

# Fixed number of rows per rolling window

In [3]:
df = pd.DataFrame({'B': [0, 1, 2, np.nan, 4]})
df

,B
0,0.000
1,1.000
2,2.000
3,nan
4,4.000


In [5]:
df.rolling??

In [9]:
df.assign(win=lambda x: x.rolling(window=2, win_type='triang').sum())

,B,win
0,0.000,nan
1,1.000,0.500
2,2.000,1.500
3,nan,nan
4,4.000,nan


In [8]:
df.assign(win=lambda x: x.rolling(window=2, win_type='gaussian').sum(std=3))

,B,win
0,0.000,nan
1,1.000,0.986
2,2.000,2.959
3,nan,nan
4,4.000,nan


In [10]:
df.assign(win=lambda x: x.rolling(window=2, win_type=None).sum())

,B,win
0,0.000,nan
1,1.000,1.000
2,2.000,3.000
3,nan,nan
4,4.000,nan


In [11]:
df.assign(win=lambda x: x.rolling(window=2, min_periods=1, win_type=None).sum())

,B,win
0,0.000,0.000
1,1.000,1.000
2,2.000,3.000
3,nan,2.000
4,4.000,4.000


# Time windows

A ragged (meaning not-a-regular frequency), time-indexed DataFrame

In [27]:
df = pd.DataFrame(
    {"B": [1, 1, 2, np.nan, 4]},
    index=[
        pd.Timestamp("20130101 09:00:00"),
        pd.Timestamp("20130101 09:00:01"),
        pd.Timestamp("20130101 09:00:05"),
        pd.Timestamp("20130101 09:00:09"),
        pd.Timestamp("20130101 09:00:10"),
    ],
)

In [28]:
df

,B
2013-01-01 09:00:00,1.000
2013-01-01 09:00:01,1.000
2013-01-01 09:00:05,2.000
2013-01-01 09:00:09,nan
2013-01-01 09:00:10,4.000


In [29]:
df.rolling('2s').sum()

,B
2013-01-01 09:00:00,1.000
2013-01-01 09:00:01,2.000
2013-01-01 09:00:05,2.000
2013-01-01 09:00:09,nan
2013-01-01 09:00:10,4.000


In [30]:
df.assign(win=lambda x: x.rolling('2s').sum())

,B,win
2013-01-01 09:00:00,1.000,1.000
2013-01-01 09:00:01,1.000,2.000
2013-01-01 09:00:05,2.000,2.000
2013-01-01 09:00:09,nan,nan
2013-01-01 09:00:10,4.000,4.000


In [31]:
df.dtypes

B    float64
dtype: object

In [32]:
df.index.dtype

dtype('<M8[ns]')

# Index of first row of window

If I have a df and I have rolling time windows, I want to know the index of the first row in the window for each row.

I will use this outside of pandas to do some aggregates.

In [64]:
tmp = (df
 .assign(idx=lambda x: range(len(x)))
 .assign(win=lambda x: x.rolling('2s')['B'].sum(),)
 .assign(win_min_idx=lambda x: x.rolling('2s')['idx'].min(),
         win_max_idx=lambda x: x.rolling('2s')['idx'].max())
)
tmp

,B,idx,win,win_min_idx,win_max_idx
2013-01-01 09:00:00,1.000,0,1.000,0.000,0.000
2013-01-01 09:00:01,1.000,1,2.000,0.000,1.000
2013-01-01 09:00:05,2.000,2,2.000,2.000,2.000
2013-01-01 09:00:09,nan,3,nan,3.000,3.000
2013-01-01 09:00:10,4.000,4,4.000,3.000,4.000
